In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [9]:
len(Precipitation_2021)
len(Precipitation_2022)

6577

In [2]:
# General simulation parameters
Nt = 24*365 # hours

In [3]:
# Law: make it rain for 50% of the simulated hours at random cumulative intervals
#      Also, randomly apply dry regions to the precipitation map

precipitation = [0]*Nt
for i in range(int(Nt/4)):
    precipitation[np.random.randint(Nt-1)] += np.random.randint(2) # this command chooses a random assigns random position in list (precipitation[argument]) value 0 or 1 (can be changed by changing the 2)

for n in range(200): #200 days will be dry with 
    random_index = np.random.randint(Nt-1-24) #values counting from zero (-1) remove a day (-24)
    for i in range(random_index, random_index+24): # make the random value +the next 24 ones all zero
        precipitation[i] = 0




In [4]:
# Reservoirs-Stocks
deep_fracture = [None]*Nt #closed L
surface_fracture = [None]*Nt #open L
quickflow= [None]*Nt #open L
tree = [None]*Nt #water sink but also a stock L

# Initial conditions
quickflow [0] = 0.1 #L
deep_fracture[0] = 1 # L
surface_fracture[0] = 4.7 # L
tree[0] = 0.2 #L

In [5]:
# Flows- Record all flow channels
q1 = [None]*Nt # precipitation -> quickflow
q2 = [None]*Nt # precipitation -> soil
q3 = [None]*Nt # soil -> fracture 
q3_1 = [None]*Nt # fracture -> soil 
q3_2 = [None]*Nt #soil -> quickflow 
q4 = [None]*Nt # fracture -> plant
q4_1 = [None]*Nt # fracture -> air
q5 = [None]*Nt # soil -> plant
q5_1 = [None]*Nt # soil -> air
q6 = [None]*Nt # plant -> air
sapflow = [None]*Nt


# Drivers and constrains 
pet = [None]*Nt
X=0.3 #concentrated fraction that is lost from P to seepage almost immediately 
sf_max=8 #maximum capacity for surface fracture
df_max=4 #maximum capacity for deep fracture 
tree_max=0.3 #maximum capacity for tree 


In [6]:
for n in range(Nt): 
    # Potential ET 
    if n % 24 == 0: 
        pet[n] = 0
    else: 
        pet[n] = np.abs(np.sin(n*(np.pi/24)))

    # Precipitation
    # Law: fraction lost to quickflow and fraction into soil 
    if precipitation[n] > 0:
        if surface_fracture[n] < sf_max:
            q1[n] = precipitation[n]*X # quickflow loss 
            q2[n] = precipitation[n]*(1-X) #infiltration into soil 
            surface_fracture[n] += q2[n]
            quickflow[n] += q1[n]
        else: 
            q1[n] = precipitation[n]*X #quickflow, the rest is assumed to be lost by overflow out of the system 
            q2 [n] = 0
    else:
        q1[n] = 0
        q2[n] = 0

    # Soil-fracture mechanism
    # Law: water flows from soil to fracture 
    #      flow rate is proportional to reservoir volume
    if surface_fracture[n]>0:
        if (surface_fracture[n] > deep_fracture[n])&(deep_fracture[n]<df_max): 
            q3[n] = 0.1*(surface_fracture[n]/sf_max)
            deep_fracture[n] += q3[n]
            surface_fracture[n] -= q3[n]
        else: 
            q3[n] = 0
    else:
        q3[n] = 0

    #Fracture-soil mechanism 
    # Law: water flows from fracture to soil at a slower rate 
    if deep_fracture[n] > 0:
        if (surface_fracture[n] < deep_fracture[n])&(surface_fracture[n]<sf_max):
            q3_1[n] = 0.0001*(deep_fracture[n]/df_max)
            surface_fracture[n] += q3_1[n]
            deep_fracture[n] -= q3_1[n]
        else: 
            q3_1[n] = 0
    else:
        q3_1[n] = 0

    # Soil to quickflow (seepage)
    # Law: by changing proportionality constant we change how connected-open are the surface fractures to open ones 
    if surface_fracture[n] > 0: #if open it'll always flow 
        q3_2[n] = 0.5*(surface_fracture[n]/sf_max)
        quickflow[n] += q3_2[n]
        surface_fracture[n] -= q3_2[n]
    else:
        q3_2[n] = 0

    #Fracture-plant mechanism
    if deep_fracture[n] >= pet[n]*(deep_fracture[n]/df_max):
        q4[n] = pet[n]*(deep_fracture[n]/df_max)
        if tree[n] < tree_max: #storage
            tree[n] += 0.05*q4[n]
            q4_1[n] = 0.95*q4[n] #ET
            deep_fracture[n] -= q4[n]   
        else: 
            q4_1[n] = q4[n] #ET
            deep_fracture[n] -= q4[n]   
    else:
        q4[n] = 0

    # Soil-plant mechanism
    if surface_fracture[n] > 0:
        q5[n] = pet[n]*(surface_fracture[n]/sf_max)
        if tree[n]<tree_max: #storage
            tree[n] += 0.05*q5[n]
            q5_1[n]= 0.95*q5[n] #ET
            surface_fracture[n] -= q5[n]
        else:
            q5_1[n]= q5[n]
            surface_fracture[n] -= q5[n]
    else:
        q5[n] = 0


    # Plant sap-flow and transpiration
    # Law: transpiration = q6
    if tree[n] <= tree_max:
        q6[n] = pet[n]*(tree[n]/tree_max)
        tree[n] -= q6[n]
    else:
        q6[n] = 0

    sapflow[n] = q4_1[n] + q5_1[n] + q6[n]


    # Copy computed results to future step
    if n < (Nt-1):
        surface_fracture[n+1] = surface_fracture[n]
        deep_fracture[n+1] = deep_fracture[n]
        quickflow [n+1] = quickflow[n]
        tree[n+1] = tree[n]
        #sapflow[n+1]=sapflow[n]
    


In [7]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=[i for i in range(Nt)], y=sapflow,
                    mode='lines',
                    name='sapflow'))
fig.add_trace(go.Scatter(x=[i for i in range(Nt)], y=tree,
                    mode='lines',
                    name='tree'))
fig.add_trace(go.Scatter(x=[i for i in range(Nt)], y=surface_fracture,
                    mode='lines',
                    name='surface'))
fig.add_trace(go.Scatter(x=[i for i in range(Nt)], y=deep_fracture,
                    mode='lines',
                    name='deep'))

fig.show()

In [8]:
assert(False)

AssertionError: 

In theory: 
- Nt/4: It might rain ~12.5% of the times only 25% of the total number of hours have a chance of being != 0 and from that it's 50-50 0 or 1 
- Nt/2: It might rain ~25% of the times only 50% of the total number of hours have a chance of being != 0 and from that it's 50-50 0 or 1 
- Nt: It might rain ~50% 50-50 0 or 1 out of all the data set 

But randomness in the position (precipitation[random]) makes the proportions be: 
- Nt/4:12-88  
- Nt/2:22-78 
- Nt:40-60

To check non zero precipitation values run this after 1st loop 

    <from collections import Counter>

    <counts=Counter(i!=0 for i in precipitation)>

    <print("rain=",counts[True],"%",(counts[True]/(counts[True]+counts[False]))*100,"No rain=",counts[False],"%",(counts[False]/(counts[True]+counts[False]))*100,"Total rain days=",counts[True]+counts[False])>